# Picking best assembly

In [1]:
WKDIR=/workspace/hraijc/Bee/BeeT/Assemblies/Purged_Assemblies
PURGEDUPS_BIN_DIR=/workspace/hraijc/git_clones/purge_dups/bin
PDDIR=${WKDIR}/purge_dups

FLYE10=/workspace/hraijc/Bee/BeeT/Assemblies/Flye_BeeT_q10/Flye_BeeT_q10.fasta
FLYE15=/workspace/hraijc/Bee/BeeT/Assemblies/Flye_BeeT_q15/Flye_BeeT_q15.fasta
HIFIASM18=/workspace/hraijc/Bee/BeeT/Assemblies/Hifiasm_q18l2kULR/BeeT_Hifiasm_q18l2kULR.asm.bp.p_ctg.fasta

ONTREADS_Q15=/workspace/hraijc/Bee/BeeT/BeeT_ONT14_1_basecalling/fastq/BeeT_Q15.fastq


In [2]:
mkdir -p ${WKDIR}/log
mkdir -p ${PDDIR}
cd ${PDDIR}

## Using Assembly_QC pipeline for contam check.

In [11]:
cd /workspace/hraijc/Bee/BeeT/Assemblies/Purged_Assemblies/assembly_qc_F10/

In [12]:
sbatch ./assembly_qc_pfr.sh

Submitted batch job 1403933


## Purge_dups

Using q15 data to purge all of the assemblies.

In [19]:
python /workspace/hraijc/git_clones/purge_dups/scripts/pd_config.py --version
python /workspace/hraijc/git_clones/purge_dups/scripts/pd_config.py --help
python /workspace/hraijc/git_clones/purge_dups/scripts/run_purge_dups.py --help


pd_config.py 0.0.0
usage: pd_config.py [-h] [-s SRF] [-l LOCD] [-B] [-n FN] [--version]
                    ref pbfofn

generate a configuration file in json format

positional arguments:
  ref                   reference file
  pbfofn                list of pacbio file (one absolute file path per line)

optional arguments:
  -h, --help            show this help message and exit
  -s SRF, --srfofn SRF  list of short reads files (one record per line, the
                        record is a tab splitted line of abosulte file path
                        plus trimmed bases, refer to
                        https://github.com/dfguan/KMC) [NONE]
  -l LOCD, --localdir LOCD
                        local directory to keep the reference and lists of the
                        pacbio, short reads files [.]
  -B, --skipB           skip running busco [False]
  -n FN, --name FN      output config file name [config.json]
  --version             show program's version number and exit
usage: run_purg

In [3]:
printf "${ONTREADS_Q15}" > ${PDDIR}/ONT_files.txt

In [4]:
python /workspace/hraijc/git_clones/purge_dups/scripts/pd_config.py -n config.FLYE10.json ${FLYE10} ${PDDIR}/ONT_files.txt
python /workspace/hraijc/git_clones/purge_dups/scripts/pd_config.py -n config.FLYE15.json ${FLYE15} ${PDDIR}/ONT_files.txt
python /workspace/hraijc/git_clones/purge_dups/scripts/pd_config.py -n config.HIFIASM18.json ${HIFIASM18} ${PDDIR}/ONT_files.txt

In [12]:
####Change json files####


In [15]:
cd ${PDDIR}

module load zlib
module load minimap2
module load BUSCO

for FILE in *.json;
do 
    PRE=${FILE%.json}
    OUTPREFIX=${PRE#config.}
    echo python /workspace/hraijc/git_clones/purge_dups/scripts/run_purge_dups.py $FILE ${PURGEDUPS_BIN_DIR} ${OUTPREFIX} -p bash;
done 

python /workspace/hraijc/git_clones/purge_dups/scripts/run_purge_dups.py config.FLYE10.json /workspace/hraijc/git_clones/purge_dups/bin FLYE10 -p bash
python /workspace/hraijc/git_clones/purge_dups/scripts/run_purge_dups.py config.FLYE15.json /workspace/hraijc/git_clones/purge_dups/bin FLYE15 -p bash
python /workspace/hraijc/git_clones/purge_dups/scripts/run_purge_dups.py config.HIFIASM18.json /workspace/hraijc/git_clones/purge_dups/bin HIFIASM18 -p bash


In [5]:

sbatch << EOF
#!/bin/bash
#SBATCH -J PurgeDups
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=24
#SBATCH --mem=30G
#SBATCH --time=11:00:00

module load zlib
module load minimap2/2.22


python /workspace/hraijc/git_clones/purge_dups/scripts/run_purge_dups.py config.FLYE10.json /workspace/hraijc/git_clones/purge_dups/bin FLYE10 -p bash
EOF

Submitted batch job 1403900


In [17]:

sbatch << EOF
#!/bin/bash
#SBATCH -J PurgeDups
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=24
#SBATCH --mem=30G
#SBATCH --time=11:00:00

module load zlib
module load minimap2/2.22


python /workspace/hraijc/git_clones/purge_dups/scripts/run_purge_dups.py config.FLYE15.json /workspace/hraijc/git_clones/purge_dups/bin FLYE15 -p bash
EOF

Submitted batch job 1403898


In [18]:

sbatch << EOF
#!/bin/bash
#SBATCH -J PurgeDups
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=24
#SBATCH --mem=30G
#SBATCH --time=11:00:00

module load zlib
module load minimap2/2.22


python /workspace/hraijc/git_clones/purge_dups/scripts/run_purge_dups.py config.HIFIASM18.json /workspace/hraijc/git_clones/purge_dups/bin HIFIASM18 -p bash
EOF

Submitted batch job 1403899


In [6]:
# Move purged assemblies
cp /workspace/hraijc/Bee/BeeT/Assemblies/Purged_Assemblies/purge_dups/BeeT_Hifiasm_q18l2kULR/seqs/BeeT_Hifiasm_q18l2kULR.asm.bp.p_ctg.purged.fa ${WKDIR}/BeeT_Hifiasm_q18l2kULR.asm.bp.p_ctg.purged.fa
cp /workspace/hraijc/Bee/BeeT/Assemblies/Purged_Assemblies/purge_dups/Flye_BeeT_q10/seqs/Flye_BeeT_q10.purged.fa ${WKDIR}/Flye_BeeT_q10.purged.fa
cp /workspace/hraijc/Bee/BeeT/Assemblies/Purged_Assemblies/purge_dups/Flye_BeeT_q15/seqs/Flye_BeeT_q15.purged.fa ${WKDIR}/Flye_BeeT_q15.purged.fa

In [7]:
cd ${WKDIR}
module load BBMap/38.33

statswrapper.sh in=BeeT_Hifiasm_q18l2kULR.asm.bp.p_ctg.purged.fa,Flye_BeeT_q10.purged.fa,Flye_BeeT_q15.purged.fa,${FLYE10},${FLYE15},${HIFIASM18} format=3


java -ea -Xmx200m -cp /software/bioinformatics/BBMap-38.33/current/ jgi.AssemblyStatsWrapper format=3 in=Flye_BeeT_q10.purged.fa format=3
n_scaffolds	n_contigs	scaf_bp	contig_bp	gap_pct	scaf_N50	scaf_L50	ctg_N50	ctg_L50	scaf_N90	scaf_L90	ctg_N90	ctg_L90	scaf_max	ctg_max	scaf_n_gt50K	scaf_pct_gt50K	gc_avg	gc_std	filename
1801	1801	684202654	684202654	0.000	77	2764729	77	2764729	286	545060	286	545060	11075509	11075509	587	98.714	0.34954	0.06735	/powerplant/workspace/hraijc/Bee/BeeT/Assemblies/Purged_Assemblies/Flye_BeeT_q10.purged.fa


| n_scaffolds 	| n_contigs 	| scaf_bp   	| contig_bp 	| gap_pct 	| scaf_N50 	| scaf_L50 	| ctg_N50 	| ctg_L50 	| scaf_N90 	| scaf_L90 	| ctg_N90 	| ctg_L90 	| scaf_max 	| ctg_max  	| scaf_n_gt50K 	| scaf_pct_gt50K 	| gc_avg  	| gc_std  	| filename                                                                                                         	|
|-------------	|-----------	|-----------	|-----------	|---------	|----------	|----------	|---------	|---------	|----------	|----------	|---------	|---------	|----------	|----------	|--------------	|----------------	|---------	|---------	|------------------------------------------------------------------------------------------------------------------	|
| 4901        	| 4901      	| 375319691 	| 375319691 	| 0.000   	| 1336     	| 92562    	| 1336    	| 92562   	| 3676     	| 42794    	| 3676    	| 42794   	| 545116   	| 545116   	| 3217         	| 84.347         	| 0.34390 	| 0.03088 	| /powerplant/workspace/hraijc/Bee/BeeT/Assemblies/Purged_Assemblies/BeeT_Hifiasm_q18l2kULR.asm.bp.p_ctg.purged.fa 	|
| 110         	| 110       	| 1259829   	| 1259829   	| 0.000   	| 14       	| 30805    	| 14      	| 30805   	| 41       	| 10014    	| 41      	| 10014   	| 67539    	| 67539    	| 4            	| 19.196         	| 0.52518 	| 0.08472 	| /powerplant/workspace/hraijc/Bee/BeeT/Assemblies/Purged_Assemblies/Flye_BeeT_q10.purged.fa                       	|
| 103         	| 103       	| 1325841   	| 1325841   	| 0.000   	| 15       	| 28060    	| 15      	| 28060   	| 52       	| 5485     	| 52      	| 5485    	| 98900    	| 98900    	| 3            	| 18.357         	| 0.53127 	| 0.07364 	| /powerplant/workspace/hraijc/Bee/BeeT/Assemblies/Purged_Assemblies/Flye_BeeT_q15.purged.fa                       	|
| 3262        	| 3262      	| 696244996 	| 696244996 	| 0.000   	| 79       	| 2696791  	| 79      	| 2696791 	| 308      	| 449130   	| 308     	| 449130  	| 11075509 	| 11075509 	| 628          	| 97.582         	| 0.35009 	| 0.06989 	| /powerplant/workspace/hraijc/Bee/BeeT/Assemblies/Flye_BeeT_q10/Flye_BeeT_q10.fasta                               	|
| 3141        	| 3141      	| 702584375 	| 702584375 	| 0.000   	| 92       	| 2088307  	| 92      	| 2088307 	| 398      	| 300382   	| 398     	| 300382  	| 13082533 	| 13082533 	| 736          	| 96.690         	| 0.35055 	| 0.07090 	| /powerplant/workspace/hraijc/Bee/BeeT/Assemblies/Flye_BeeT_q15/Flye_BeeT_q15.fasta                               	|
| 4993        	| 4993      	| 380881427 	| 380881427 	| 0.000   	| 1364     	| 92058    	| 1364    	| 92058   	| 3749     	| 42794    	| 3749    	| 42794   	| 545116   	| 545116   	| 3273         	| 84.215         	| 0.34380 	| 0.03146 	| /powerplant/workspace/hraijc/Bee/BeeT/Assemblies/Hifiasm_q18l2kULR/BeeT_Hifiasm_q18l2kULR.asm.bp.p_ctg.fasta     	|

In [ ]:
1801	1801	684202654	684202654	0.000	77	2764729	77	2764729	286	545060	286	545060	11075509	11075509	587	98.714	0.34954	0.06735	/powerplant/workspace/hraijc/Bee/BeeT/Assemblies/Purged_Assemblies/Flye_BeeT_q10.purged.fa
